## Natural Language Processing - Assignment 2
#### November 2018

###### Index Number: 14000334

### Answer (a)

- Both negative and positive reviews read as two dataframes using pandas
- Dataframes are concatenate the positive and negative review to one dataframe
- Train and test dataframes are written into two CSV files

In [1]:
import pandas as pd
from glob import glob
import warnings

# Removing future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

negReviews = 12500
posReviews = 12500

# Reading the negative reviews train data
negData = []
negDataDir = 'aclImdb/train/neg/'
for i in range(negReviews):
    fileName = glob(negDataDir+str(i)+'_*.txt')[0]
    f = open(fileName, "r", encoding="utf8")
    lines = f.readlines()
    negData.append([lines[0],'0'])
negDataFrame = pd.DataFrame(negData,columns=['review','sentiment'])

# Reading the positive reviews train data
posData = []
posDataDir = 'aclImdb/train/pos/'
for i in range(posReviews):
    fileName = glob(posDataDir+str(i)+'_*.txt')[0]
    f = open(fileName, "r", encoding="utf8")
    lines = f.readlines()
    posData.append([lines[0],'1'])
posDataFrame = pd.DataFrame(posData,columns=['review','sentiment'])

trainDataFrame = pd.concat([posDataFrame,negDataFrame])

# Writting the Train data into a CSV
trainDataFrame.to_csv('train.csv', sep=',', encoding='utf-8', index=False)

# Reading the negative reviews test data
negData = []
negDataDir = 'aclImdb/test/neg/'
for i in range(negReviews):
    fileName = glob(negDataDir+str(i)+'_*.txt')[0]
    f = open(fileName, "r", encoding="utf8")
    lines = f.readlines()
    negData.append([lines[0],'0'])
negDataFrame = pd.DataFrame(negData,columns=['review','sentiment'])

# Reading the positive reviews test data
posData = []
posDataDir = 'aclImdb/test/pos/'
for i in range(posReviews):
    fileName = glob(posDataDir+str(i)+'_*.txt')[0]
    f = open(fileName, "r", encoding="utf8")
    lines = f.readlines()
    posData.append([lines[0],'1'])
posDataFrame = pd.DataFrame(posData,columns=['review','sentiment'])

testDataFrame = pd.concat([posDataFrame,negDataFrame])

# Writting the Test data into a CSV
testDataFrame.to_csv('test.csv', sep=',', encoding='utf-8', index=False)
print("Dataset Created\n")
print("Train Data Size: %d\nTest Data size: %d\n" % ((negReviews+posReviews),(negReviews+posReviews)))

Dataset Created

Train Data Size: 25000
Test Data size: 25000



12500 negative reviews and 12500 positives reviews are used to create the train dataset <br>
Same number of reviews are used to create the test dataset <br>
Train data: <b>25000 reviews </b><br>
Test data: <b>25000 reviews</b>

#### ------------------------------------------------------------

### Answer (b)

- Loaded the data from the CSV files to dataframes
- Removed the accented charaters using 'unidecode'
- Unescaping HTML escape sequences
- Removed html tags using 'BeautifulSoup'
- Expanded language contractions using custom contraction library
- Removed Special characters
- Removed words that present in less than five reviews
- Cleaned reviews are written into CSV files

In [2]:
import unidecode
import html
import re
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer

contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he has",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how does",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I shall",
    "I'll've": "I shall have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it has",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she has",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

# Expanding contractions
contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
def expandContractions(s, contractions=contractions):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, s)

# removing special characters (removed all characters except the alphabet)
def removeSpecialChars(review):
    return re.sub('[^A-Za-z ]', '', review)

# cleaning the reviews
def cleanReview(review):
    # Normalizing accented characters
    review = unidecode.unidecode(review)
    # Unescaping HTML escape sequences
    review = html.unescape(review)
    # Removing extraneous html tags
    review = BeautifulSoup(review).get_text()
    # Expanding contractions
    review = expandContractions(review)
    # Removing special characters
    review = removeSpecialChars(review)
    # return the cleaned review
    return review

# return the words larger than the given frequency
def getTokens(reviews, freq=0):
    vec = CountVectorizer().fit((reviews).values.astype('U'))
    bag_of_words = vec.transform(reviews)
    sum_words = bag_of_words.sum(axis=0) 
    tokens = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    tokens =sorted(tokens, key = lambda x: x[1], reverse=True)
    tokens = [token for (token, frequency) in tokens if frequency > freq]
    return tokens

# return review by removing words that present in less than five reviews
def rmvLessFreqTokens(review, freqList):
    result = ' '.join(list(filter(lambda x: x in freqList, review.split(' '))))
    if result == '':
        result = 'review'
    return result
    

# Loading csv files data into dataframes
trainDataFrame = pd.read_csv('train.csv', sep=',')
testDataFrame = pd.read_csv('test.csv', sep=',')

# cleaning the train and test reviews
try:
    trainDataFrame['review'] = trainDataFrame['review'].apply(lambda review:cleanReview(review))
except:
    pass
try:
    testDataFrame['review'] = testDataFrame['review'].apply(lambda review:cleanReview(review))
except:
    pass
    
# Counting the number of training and testing data set tokens   
trainTokens = getTokens(trainDataFrame['review'],0)
testTokens = getTokens(testDataFrame['review'],0)

print('Number of Tokens:')
print('\tTraining:', len(trainTokens))
print('\tTesting:', len(testTokens))

# getting tokens that have frequency more than 5 in training data set
trainTokens = getTokens(trainDataFrame['review'],5)
# getting tokens that have frequency more than 5 in testing data set
testTokens = getTokens(testDataFrame['review'],5)

# Removing the less frequent words from the train and test data sets
try:
    trainDataFrame['review'] = trainDataFrame['review'].apply(lambda review:rmvLessFreqTokens(review,trainTokens))
except:
    pass
try:
    testDataFrame['review'] = testDataFrame['review'].apply(lambda review:rmvLessFreqTokens(review,testTokens))
except:
    pass
    
# Counting the number of training and testing data set tokens   
trainTokens = getTokens(trainDataFrame['review'],0)
testTokens = getTokens(testDataFrame['review'],0)

print('\nNumber of Tokens(removed less frequent words):')
print('\tTraining:', len(trainTokens))
print('\tTesting:', len(testTokens))


# Writting the processed dataframes to CSV files
trainDataFrame.to_csv('trainDataset1.csv', sep=',', encoding='utf-8', index=False)
testDataFrame.to_csv('testDataset1.csv', sep=',', encoding='utf-8', index=False)


Number of Tokens:
	Training: 137388
	Testing: 134517

Number of Tokens(removed less frequent words):
	Training: 22011
	Testing: 21811


Number of tokens reduced after processing the dataset

#### ------------------------------------------------------------

### Answer (c)

- Loaded data from CSV files
- Removed stopwords from the reviews
- Created second dataset by removing stop words
- Lemmatizating the data
- Created third dataset
- Create separate SVM for three datasets 

In [15]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.exceptions import ConvergenceWarning
import pandas as pd
import numpy as np

# Removing convergence warnings
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)


# getting the stop words
stopWords = set(stopwords.words('english'))

# Reduce the noice of the given review
def reduceNoice(review):
    # tokenize the input
    tokens = word_tokenize(review)
    # removing the stop words
    filteredTokens = [word for word in tokens if not word in stopWords] 
    # converting the token array into a string
    filteredTokens = ' '.join(filteredTokens)
    return filteredTokens

lemmatizer = WordNetLemmatizer()
def lemmatize(review):
    # tokenize the input
    tokens = word_tokenize(review)
    # lemmatizing
    lemmatizedTokens = [lemmatizer.lemmatize(word) for word in tokens] 
    lemmatizedTokens = ' '.join(lemmatizedTokens)
    return lemmatizedTokens

# return the words larger than the given frequency
def getTokens(reviews, freq=0):
    vec = CountVectorizer().fit((reviews).values.astype('U'))
    bag_of_words = vec.transform(reviews)
    sum_words = bag_of_words.sum(axis=0) 
    tokens = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    tokens =sorted(tokens, key = lambda x: x[1], reverse=True)
    tokens = [token for (token, frequency) in tokens if frequency > freq]
    return tokens

# Loading csv files data into dataframes
trainDataFrame = pd.read_csv('trainDataset1.csv', sep=',')
testDataFrame = pd.read_csv('testDataset1.csv', sep=',')
    
# Adding a new dataframe  
trainDataFrame2 = pd.DataFrame([],columns=['review'])
trainDataFrame2['review'] = trainDataFrame['review']

testDataFrame2 = pd.DataFrame([],columns=['review'])
testDataFrame2['review'] = testDataFrame['review']

# reducing the noice train and test reviews
try:
    trainDataFrame2['review'] = trainDataFrame2['review'].apply(lambda review:reduceNoice(review))
except:
    pass
try:
    testDataFrame2['review'] = testDataFrame2['review'].apply(lambda review:reduceNoice(review))
except:
    pass

# Writting the processed dataframes to CSV files
trainDataFrame2.to_csv('trainDataset2.csv', sep=',', encoding='utf-8', index=False)
testDataFrame2.to_csv('testDataset2.csv', sep=',', encoding='utf-8', index=False)


# Adding a new dataframe  
trainDataFrame3 = pd.DataFrame([],columns=['review'])
trainDataFrame3['review'] = trainDataFrame2['review']

testDataFrame3 = pd.DataFrame([],columns=['review'])
testDataFrame3['review'] = testDataFrame2['review']

# stemming and lemmatizing the noice train and test reviews
try:
    trainDataFrame3['review'] = trainDataFrame3['review'].apply(lambda review:lemmatize(review))
except:
    pass
try:
    testDataFrame3['review'] = testDataFrame3['review'].apply(lambda review:lemmatize(review))
except:
    pass


# Writting the processed dataframes to CSV files
trainDataFrame3.to_csv('trainDataset3.csv', sep=',', encoding='utf-8', index=False)
testDataFrame3.to_csv('testDataset3.csv', sep=',', encoding='utf-8', index=False)

# Counting the number of training and testing data set tokens   
trainTokens = getTokens(trainDataFrame['review'],0)
testTokens = getTokens(testDataFrame['review'],0)

print('\nNumber of Tokens(removed less frequent words):')
print('\tTraining:', len(trainTokens))
print('\tTesting:', len(testTokens))

trainTokens = getTokens(trainDataFrame2['review'],0)
testTokens = getTokens(testDataFrame2['review'],0)

print('\nNumber of Tokens(stop words removed):')
print('\tTraining:', len(trainTokens))
print('\tTesting:', len(testTokens))

trainTokens = getTokens(trainDataFrame3['review'],0)
testTokens = getTokens(testDataFrame3['review'],0)

print('\nNumber of Tokens(lemmatized):')
print('\tTraining:', len(trainTokens))
print('\tTesting:', len(testTokens))

trainDataFrames = [trainDataFrame,trainDataFrame2,trainDataFrame3]
testDataFrames = [testDataFrame,testDataFrame2,testDataFrame3]

# Generating SVMs and evaluating the models
print("\n*** Results from the models ***\n")
for datasetIndx in range(len(trainDataFrames)):
    print("Dataset %d:"%(datasetIndx+1))
    vectorizer = CountVectorizer() 
    vectorizer.fit(trainDataFrames[datasetIndx]['review'].values.astype('U'))
    # vectorizing the training set
    trainData = vectorizer.transform(trainDataFrames[datasetIndx]['review'].values.astype('U'))
    # vectorizing the testing set
    testData = vectorizer.transform(testDataFrames[datasetIndx]['review'])
    # training the classifier
    classifier = svm.LinearSVC()
    classifier.fit(trainData, trainDataFrame['sentiment'])
    # cross validation
    scores = cross_val_score(classifier, trainData, trainDataFrame['sentiment'], cv=5, n_jobs=-1)
    print("\tCross Validation Scores:",scores)
    print("\tModel Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    # testing 
    predictionScore = classifier.score(testData,testDataFrame['sentiment'])
    print("\tTest Accuracy: %0.2f\n" % predictionScore)


Number of Tokens(removed less frequent words):
	Training: 22011
	Testing: 21811

Number of Tokens(stop words removed):
	Training: 21880
	Testing: 21679

Number of Tokens(lemmatized):
	Training: 18773
	Testing: 18619

*** Results from the models ***

Dataset 1:
	Cross Validation Scores: [0.8276 0.8342 0.8264 0.834  0.8342]
	Model Accuracy: 0.83 (+/- 0.01)
	Test Accuracy: 0.84

Dataset 2:
	Cross Validation Scores: [0.831  0.8276 0.8248 0.8282 0.8304]
	Model Accuracy: 0.83 (+/- 0.00)
	Test Accuracy: 0.84

Dataset 3:
	Cross Validation Scores: [0.8286 0.8358 0.8164 0.8212 0.832 ]
	Model Accuracy: 0.83 (+/- 0.01)
	Test Accuracy: 0.83



<b>Model 1:</b> Words that present in less than five reviews removed (Dataset 1)  <br>
<b>Model 2:</b> Stop words removed (Dataset 2) <br>
<b>Model 3:</b> Lemmatized (Dataset 3) <br><br>

No significance difference between cross validation scores and the test accuracies of the 3 models <br>
SVM models 1 and 2 have same accuracies <br>
Accuracy: <b>84%</b> <br>
Model 3 has low accuracy compared to other two models <br>
Accuracy: <b>83%</b>   
</b>
#### ------------------------------------------------------------

### Answer (d)
- SVM model using tf-idf values

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

print("\n*** Using TFIDF values ***\n")
datasetIndx = 2
vectorizer = TfidfVectorizer()
vectorizer.fit(trainDataFrames[datasetIndx]['review'].values.astype('U'))
# vectorizing the training set
trainData = vectorizer.transform(trainDataFrames[datasetIndx]['review'].values.astype('U'))
# vectorizing the testing set
testData = vectorizer.transform(testDataFrames[datasetIndx]['review'])
# training the classifier
classifier = svm.LinearSVC()
classifier.fit(trainData, trainDataFrame['sentiment'])
# cross validation
scores = cross_val_score(classifier, trainData, trainDataFrame['sentiment'], cv=5, n_jobs=-1)
print("Cross Validation Scores:",scores)
print("Model Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# testing 
predictionScore = classifier.score(testData,testDataFrame['sentiment'])
print("Test Accuracy: %0.2f\n" % predictionScore)


*** Using TFIDF values ***

Cross Validation Scores: [0.8584 0.8634 0.8484 0.8564 0.8616]
Model Accuracy: 0.86 (+/- 0.01)
Test Accuracy: 0.87



Used term frequency–inverse document frequency values <br>
Cross validation scores are in the range <b> 0.85 - 0.86 </b> <br>
No significance difference between cross validation scores and the test accuracies of the 3 models <br>
Model with highest accuray when compared to all the models <br>
Accuracy: <b>87%</b>   

#### ------------------------------------------------------------

### Answer (e)

- SVM Using tf-idf values and bigrams

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

print("\n*** Using TFIDF values and bigrams ***\n")
datasetIndx = 2
vectorizer = TfidfVectorizer(min_df=1, ngram_range=(2, 2))
vectorizer.fit(trainDataFrames[datasetIndx]['review'].values.astype('U'))
# vectorizing the training set
trainData = vectorizer.transform(trainDataFrames[datasetIndx]['review'].values.astype('U'))
# vectorizing the testing set
testData = vectorizer.transform(testDataFrames[datasetIndx]['review'])
# training the classifier
classifier = svm.LinearSVC()
classifier.fit(trainData, trainDataFrame['sentiment'])
# cross validation
scores = cross_val_score(classifier, trainData, trainDataFrame['sentiment'], cv=5, n_jobs=-1)
print("Cross Validation Scores:",scores)
print("Model Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# testing 
predictionScore = classifier.score(testData,testDataFrame['sentiment'])
print("Test Accuracy: %0.2f\n" % predictionScore)


*** Using TFIDF values and bigrams ***

Cross Validation Scores: [0.825  0.825  0.8226 0.826  0.8296]
Model Accuracy: 0.83 (+/- 0.00)
Test Accuracy: 0.84



Used term frequency–inverse document frequency (tf-idf) values with bigrams <br>
No significance difference between cross validation score and the test accuracy <br>
Accuracy is low compared to SVM model with tf-idf with unigrams  <br>
Accuracy is same as the SVM with bag-of-words feature model  <br>
Accuracy: <b>84%</b> <br><br>

- <b> Model Accuracies </b>

<table align="left">
  <tr>
    <th>SVM Model</th>
    <th>Features</th>
    <th>Cross Validation</th>
    <th>Test Accuracy</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Bag Of Words (cleaned)</td>
    <td>.83</td>
    <td>.84</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Bag Of Words (stop words removed)</td>
    <td>.83</td>
    <td>.84</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Bag Of Words (lemmatized)</td>
    <td>.83</td>
    <td>.83</td>
  </tr>
  <tr>
    <td><b>4</b></td>
    <td><b>TF-IDF with unigrams</b></td>
    <td><b>.86</b></td>
    <td><b>.87</b></td>
  </tr>
  <tr>
    <td>5</td>
    <td>TF-IDF with bigrams</td>
    <td>.83</td>
    <td>.84</td>
  </tr>
</table>


<br><br><br><br><br><br><br><br><br>

#### ------------------------------------------------------------

### Answer (f)

- Logistic regression for the third dataset

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

print("\n*** Using Logistic regression ***\n")

datasetIndx = 2
vectorizer = CountVectorizer(ngram_range=(1, 1))
vectorizer.fit(trainDataFrames[datasetIndx]['review'].values.astype('U'))
# vectorizing the training set
trainData = vectorizer.transform(trainDataFrames[datasetIndx]['review'].values.astype('U'))
# vectorizing the testing set
testData = vectorizer.transform(testDataFrames[datasetIndx]['review'])

classifier = LogisticRegression()
# Train the model using default parameters
classifier.fit(trainData, trainDataFrame['sentiment'])
# cross validation
scores = cross_val_score(classifier, trainData, trainDataFrame['sentiment'], cv=5, n_jobs=-1)
print("Cross Validation Scores:",scores)
print("Model Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# testing 
predictionScore = classifier.score(testData,testDataFrame['sentiment'])
print("Test Accuracy: %0.2f\n" % predictionScore)


*** Using Logistic regression ***

Cross Validation Scores: [0.848  0.8508 0.8398 0.85   0.853 ]
Model Accuracy: 0.85 (+/- 0.01)
Test Accuracy: 0.86



Used Logistic Regression model with unigrams <br>
No significance difference between cross validation score and the test accuracy <br>
Second highest accuracy, only less than the SVM model with tf-idf with unigrams <br>
Accuracy: <b>86%</b>

#### ------------------------------------------------------------